In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.models import load_model

Using TensorFlow backend.


In [2]:
model = load_model('./models/best_model_1000.h5')

C:\Users\boinit\anaconda3\envs\myenv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [2]:
test= pd.read_csv('new_samples.csv',header=None)
test.columns=['Math Expresion','Expression Name']

In [3]:
test

,Math Expresion,Expression Name
0,6+\frac{5}{3} x=0,linear_equation
1,-8\left(-7x-1-\frac{8}{5} x+4\right),distributor_expression
2,\frac{5}{9} \left(\frac{1}{3} x+\frac{4}{3}+7....,distributor_expression
3,-9+7x+8x-7.3=1.6\left(\frac{-4}{7} x+\frac{7}{...,linear_equation
4,\frac{-1}{7} \left(\frac{-4}{9} x+1\right)+0.2...,distributor_expression
...,...,...
95,8.5\left(5x+\frac{1}{2}\right),distributor_expression
96,\frac{3}{5}-6.2x=-0.4x+3,linear_equation
97,\frac{5}{3} \left(-0.4x+\frac{5}{8}+1+7x\right),distributor_expression
98,4.2\left(-8x-8.5-6.3x+9\right)=-2+6.1x+8.9x+\f...,linear_equation


In [5]:
#test = test_original.iloc[80000:81000,].reset_index(drop=True).copy()


In [4]:
test_ME = test.iloc[:,0].reset_index(drop=True)
test_target = test.iloc[:,1].reset_index(drop=True)

In [5]:
test.loc[test["Expression Name"]=='distributor_expression','Expression Name']=0
test.loc[test["Expression Name"]=='linear_expression','Expression Name']=1
test.loc[test["Expression Name"]=='linear_equation','Expression Name']=2

In [6]:
test

,Math Expresion,Expression Name
0,6+\frac{5}{3} x=0,2
1,-8\left(-7x-1-\frac{8}{5} x+4\right),0
2,\frac{5}{9} \left(\frac{1}{3} x+\frac{4}{3}+7....,0
3,-9+7x+8x-7.3=1.6\left(\frac{-4}{7} x+\frac{7}{...,2
4,\frac{-1}{7} \left(\frac{-4}{9} x+1\right)+0.2...,0
...,...,...
95,8.5\left(5x+\frac{1}{2}\right),0
96,\frac{3}{5}-6.2x=-0.4x+3,2
97,\frac{5}{3} \left(-0.4x+\frac{5}{8}+1+7x\right),0
98,4.2\left(-8x-8.5-6.3x+9\right)=-2+6.1x+8.9x+\f...,2


In [7]:
a=0
for s in test.iloc[:,0]:
    test.iloc[a,0] = bytearray(s, 'utf-8')
    a+=1
    #print(test.iloc[a,0][0]+'  {}'.format(a))

In [9]:
test_X_test = test.iloc[:,0]
test_y_test = test.iloc[:,1]

In [10]:
X_test = sequence.pad_sequences(test_X_test, maxlen=200)

In [11]:
Y_test= np_utils.to_categorical(test_y_test)

In [12]:
##print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

In [13]:
pre = model.predict(X_test)
pre_list=[]
for i in range(len(pre)) :
    if np.argmax(pre[i]) == 0 :
        pre_list.append('distributor_expression')
    elif np.argmax(pre[i]) == 1 :
        pre_list.append('linear_expression')
    elif np.argmax(pre[i]) == 2 :
        pre_list.append('linear_equation')

In [14]:
acc = []
for i in range(len(pre)) :
    if pre_list[i] == test_target[i] :
        acc.append('O')
    else :
        acc.append('X')
    

In [15]:
result = pd.DataFrame(data={'원본수식' : test_ME,'정답' : test_target, '예측값' : pre_list, '일치여부' : acc},
                      columns=['원본수식',"예측값",'정답','일치여부'],index=None)

In [16]:
result=result.reset_index(drop=True)


In [17]:
result

,원본수식,예측값,정답,일치여부
0,6+\frac{5}{3} x=0,linear_equation,linear_equation,O
1,-8\left(-7x-1-\frac{8}{5} x+4\right),distributor_expression,distributor_expression,O
2,\frac{5}{9} \left(\frac{1}{3} x+\frac{4}{3}+7....,distributor_expression,distributor_expression,O
3,-9+7x+8x-7.3=1.6\left(\frac{-4}{7} x+\frac{7}{...,linear_equation,linear_equation,O
4,\frac{-1}{7} \left(\frac{-4}{9} x+1\right)+0.2...,linear_equation,distributor_expression,X
...,...,...,...,...
95,8.5\left(5x+\frac{1}{2}\right),distributor_expression,distributor_expression,O
96,\frac{3}{5}-6.2x=-0.4x+3,linear_equation,linear_equation,O
97,\frac{5}{3} \left(-0.4x+\frac{5}{8}+1+7x\right),distributor_expression,distributor_expression,O
98,4.2\left(-8x-8.5-6.3x+9\right)=-2+6.1x+8.9x+\f...,linear_equation,linear_equation,O


In [18]:
error_index=[]
if acc.count('X')>0 :
    for i in range(len(acc)):
        if acc[i] == "X":
            error_index.append(i)
for i in error_index :
    print("index : "+str(i))
    print("원본수식 :" + result.iloc[i,0]+ '\n'+"예측값 : " + result.iloc[i,1] + '\n'+"정답 : " + result.iloc[i,2]+"\n")
print()
print("정확도 : {}%, 테스트 {}문항 중 {}문항 예측실패".
      format(((len(pre)-len(error_index))/len(pre))*100,len(pre),len(error_index)))

index : 4
원본수식 :\frac{-1}{7} \left(\frac{-4}{9} x+1\right)+0.2\left(6+6.8x-\frac{5}{2} x+8.2\right)
예측값 : linear_equation
정답 : distributor_expression

index : 13
원본수식 :-3.7+4x-8x-\frac{1}{4}=-8\left(7x-\frac{1}{2}\right)
예측값 : distributor_expression
정답 : linear_equation

index : 19
원본수식 :7x+6=-5.8+6.9x
예측값 : linear_expression
정답 : linear_equation

index : 20
원본수식 :\frac{-7}{4} x+7.8-\frac{6}{7} x-4.5
예측값 : linear_equation
정답 : linear_expression

index : 28
원본수식 :-0.8\left(-5.9x+3.4+\frac{3}{8} x+2.3\right)-6\left(-9x+6.8+3x+5.2\right)
예측값 : linear_equation
정답 : distributor_expression

index : 32
원본수식 :-8.4x+\frac{9}{8}=\frac{1}{3} x-\frac{1}{4}
예측값 : linear_expression
정답 : linear_equation

index : 33
원본수식 :-5-2.6x=4x+9
예측값 : linear_expression
정답 : linear_equation

index : 36
원본수식 :-3.1\left(7.4+3x+\frac{1}{9} x-5\right)+6.6\left(\frac{-3}{5}-\frac{1}{4} x\right)
예측값 : linear_equation
정답 : distributor_expression

index : 41
원본수식 :-6\left(-5.1+7x-\frac{1}{2}+\frac{4}{9} x\right)+\frac{3}

In [19]:
result.to_csv("result.csv",encoding='utf-8-sig')